In [1]:
import elli

In [2]:
MM = elli.SpectraRay.read_mmatrix('Wafer_MM_70.txt').loc[210:820]

In [3]:
params = elli.ParamsHist()
params.add('SiO2_n0', value=1.452, min=-100, max=100, vary=True)
params.add('SiO2_n1', value=36.0, min=-40000, max=40000, vary=True)
params.add('SiO2_n2', value=0, min=-40000, max=40000, vary=True)
params.add('SiO2_k0', value=0, min=-100, max=100, vary=True)
params.add('SiO2_k1', value=0, min=-40000, max=40000, vary=True)
params.add('SiO2_k2', value=0, min=-40000, max=40000, vary=True)
params.add('SiO2_d', value=120, min=0, max=40000, vary=True)

In [4]:
@elli.fit_mueller_matrix(MM, params, display_single=False, sharex=True, full_scale=False)
def model(lbda, params):
    sr = elli.SpectraRay('./')
    Si = elli.IsotropicMaterial(sr.loadDispersionTable('Si_Aspnes.mat'))

    SiO2 = elli.IsotropicMaterial(elli.DispersionCauchy(params['SiO2_n0'],
                                                    params['SiO2_n1'],
                                                    params['SiO2_n2'],
                                                    params['SiO2_k0'],
                                                    params['SiO2_k1'],
                                                    params['SiO2_k2']))

    Layer = [elli.Layer(SiO2, params['SiO2_d'])]

    return elli.Structure(elli.AIR, Layer, Si).evaluate(lbda, 70, solver=elli.Solver4x4, propagator=elli.PropagatorEig())

In [69]:
model.fit()

In [5]:
model.plot(full_scale=False)

FigureWidget({
    'data': [{'line': {'color': '#636EFA', 'dash': 'solid'},
              'name': 'M11 ',
    …